In [ ]:
!pip install transformers datasets accelerate


In [4]:
from datasets import load_dataset

dataset = load_dataset("text", data_files={"train": "dataset.txt"})


Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [12]:
def tokenize_and_add_labels(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize_and_add_labels, batched=False)


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [11]:
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=500,
    warmup_steps=50,
    report_to="none"  # <-- This disables wandb, tensorboard, everything
)


In [14]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset

# load dataset
dataset = load_dataset("text", data_files={"train": "dataset.txt"})

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_and_add_labels(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize_and_add_labels, batched=False)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=200,
    warmup_steps=50,
    report_to="none"   # disables wandb
)

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"]
)

trainer.train()


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=9, training_loss=8.306686401367188, metrics={'train_runtime': 95.5774, 'train_samples_per_second': 0.094, 'train_steps_per_second': 0.094, 'total_flos': 587907072000.0, 'train_loss': 8.306686401367188, 'epoch': 3.0})

In [15]:
from transformers import pipeline

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

generator("Once upon a time", max_length=100)


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': 'Once upon a time, the last known time something happened in our world that occurred.\n\nThe first of these is the "Black" time event that occurs in the "Black" universe, and the last known time event that occurs in the "Black universe.\n\nThe "Black" Universe\n\nThe Black universe universe is the "Black" universe of all galaxies.\n\nThe "Black" Universe\n\nThe "Black" universe universe is the "Black" universe of all galaxies.\n\nThe "Black" Universe\n\nThe "Black" Universe\n\nThe Black universe universe is the "Black" universe of all galaxies, and also of all stars.\n\nThe "Black" Universe"\n\nThe "Black" universe is the "Black" universe of all galaxies and stars.\n\nThe "Black" Universe\n\nThe "Black" universe is the "Black" universe of all of galaxies and stars.\n\nThe "Black" Universe\n\nThe "Black" universe is the "Black" universe of all galaxies, and also of all stars.\n\nThe "Black" Universe\n\nThe "Black" universe is the "Black" universe of all galaxies and 